# Develop and test hash based node storage

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path(Path.cwd()).parent))

In [2]:
import warnings
warnings.filterwarnings("ignore")

%config IPCompleter.evaluation='unsafe'

In [3]:
from node_library.development import hash_based_storage as hs

### Perform a few tests for the hash_based_storage module

In [4]:
db = hs.create_nodes_table(echo=False)

In [5]:
hs.list_column_names(db, 'node')

['node_id',
 'name',
 'hash_value',
 'lib_path',
 'creation_date',
 'data',
 'output_ready',
 'file_path']

**Note:** Uncomment the following line if you have modified the table structure. After deleting the table comment this block and rerun the notebook (to run create_nodes_table)

In [6]:
# hs.drop_table(db, 'node')
# hs.list_column_names(db, 'node')

In [7]:
hs.list_table(db)

,node_id,name,hash_value,lib_path,creation_date,data,output_ready,file_path
0,2,,da5ce3a27254d1f118ad19a64728e063f5508e53870da3...,node_library/math/Sin,2024-06-15 09:54:25.239067,{'x': '[8]'},False,
1,3,,ad5107dc887ec851fadc22a9d63ec7bfafa518f79ec3d7...,node_library/math/Sin,2024-06-15 09:57:19.661584,{'x': '[7]'},True,node_library/math/Sin/3
2,4,,c16d79b83fad085c61d52223687f65ac5f258005921c83...,node_library/math/Sin,2024-06-15 12:51:12.179914,{'x': '[17]'},True,node_library/math/Sin/4
3,5,,627be35de16dc8fa1bbb384f46cf9a63c1843d7d24b51d...,node_library/math/Sin,2024-06-15 13:05:02.263759,{'x': '[11]'},True,node_library/math/Sin/5


In [8]:
hs.remove_nodes_from_db(db, indices=[1, 15]);

In [9]:
df = hs.list_table(db)
df

,node_id,name,hash_value,lib_path,creation_date,data,output_ready,file_path
0,2,,da5ce3a27254d1f118ad19a64728e063f5508e53870da3...,node_library/math/Sin,2024-06-15 09:54:25.239067,{'x': '[8]'},False,
1,3,,ad5107dc887ec851fadc22a9d63ec7bfafa518f79ec3d7...,node_library/math/Sin,2024-06-15 09:57:19.661584,{'x': '[7]'},True,node_library/math/Sin/3
2,4,,c16d79b83fad085c61d52223687f65ac5f258005921c83...,node_library/math/Sin,2024-06-15 12:51:12.179914,{'x': '[17]'},True,node_library/math/Sin/4
3,5,,627be35de16dc8fa1bbb384f46cf9a63c1843d7d24b51d...,node_library/math/Sin,2024-06-15 13:05:02.263759,{'x': '[11]'},True,node_library/math/Sin/5


In [10]:
hs.transform_data_column(df)

,node_id,name,hash_value,lib_path,creation_date,output_ready,file_path,x
0,2,,da5ce3a27254d1f118ad19a64728e063f5508e53870da3...,node_library/math/Sin,2024-06-15 09:54:25.239067,False,,[8]
1,3,,ad5107dc887ec851fadc22a9d63ec7bfafa518f79ec3d7...,node_library/math/Sin,2024-06-15 09:57:19.661584,True,node_library/math/Sin/3,[7]
2,4,,c16d79b83fad085c61d52223687f65ac5f258005921c83...,node_library/math/Sin,2024-06-15 12:51:12.179914,True,node_library/math/Sin/4,[17]
3,5,,627be35de16dc8fa1bbb384f46cf9a63c1843d7d24b51d...,node_library/math/Sin,2024-06-15 13:05:02.263759,True,node_library/math/Sin/5,[11]


In [11]:
hs.db_query_dict(db, x='[1]')

""


### Connect to pyiron_workflow

In [12]:
from pyiron_workflow import Workflow

In [13]:
# Register the necessary node packages                                                                        
Workflow.register("node_library.math", "math")

In [14]:
sin = Workflow.create.math.Sin(x=[17])
sin.outputs.sin.value

NOT_DATA

In [15]:
df = hs.list_table(db)
df

,node_id,name,hash_value,lib_path,creation_date,data,output_ready,file_path
0,2,,da5ce3a27254d1f118ad19a64728e063f5508e53870da3...,node_library/math/Sin,2024-06-15 09:54:25.239067,{'x': '[8]'},False,
1,3,,ad5107dc887ec851fadc22a9d63ec7bfafa518f79ec3d7...,node_library/math/Sin,2024-06-15 09:57:19.661584,{'x': '[7]'},True,node_library/math/Sin/3
2,4,,c16d79b83fad085c61d52223687f65ac5f258005921c83...,node_library/math/Sin,2024-06-15 12:51:12.179914,{'x': '[17]'},True,node_library/math/Sin/4
3,5,,627be35de16dc8fa1bbb384f46cf9a63c1843d7d24b51d...,node_library/math/Sin,2024-06-15 13:05:02.263759,{'x': '[11]'},True,node_library/math/Sin/5


In [16]:
hs.save_node(sin, db) 

save path:  /Users/joerg/python_projects/git_libs/pyiron_nodes/notebooks/node_library/math/Sin/4/project.h5


4

In [17]:
sin2 = hs.get_node_from_db_id(3, db)
sin2.inputs.x.value, sin2.outputs.sin.value

([7], array([0.6569866]))

In [18]:
hs.list_table(db)

,node_id,name,hash_value,lib_path,creation_date,data,output_ready,file_path
0,2,,da5ce3a27254d1f118ad19a64728e063f5508e53870da3...,node_library/math/Sin,2024-06-15 09:54:25.239067,{'x': '[8]'},False,
1,3,,ad5107dc887ec851fadc22a9d63ec7bfafa518f79ec3d7...,node_library/math/Sin,2024-06-15 09:57:19.661584,{'x': '[7]'},True,node_library/math/Sin/3
2,4,,c16d79b83fad085c61d52223687f65ac5f258005921c83...,node_library/math/Sin,2024-06-15 12:51:12.179914,{'x': '[17]'},True,node_library/math/Sin/4
3,5,,627be35de16dc8fa1bbb384f46cf9a63c1843d7d24b51d...,node_library/math/Sin,2024-06-15 13:05:02.263759,{'x': '[11]'},True,node_library/math/Sin/5


In [19]:
sin = Workflow.create.math.Sin(x=[12])
sin12 = hs.run_node(sin, db, verbose=True)

save path:  /Users/joerg/python_projects/git_libs/pyiron_nodes/notebooks/node_library/math/Sin/6/project.h5


In [20]:
sin

In [21]:
sin.outputs.sin

In [24]:
for node_id in range(8):
    sin2 = hs.get_node_from_db_id(node_id, db)
    if sin2 is not None:
        print (f'Node id: {node_id}, inputs: {sin2.inputs.x.value}, outputs: {sin2.outputs.sin.value}')
    else:
        print (f'Node id: {node_id} does not exist')

Node id: 0 does not exist
Node id: 1 does not exist
Node id: 2, inputs: [8], outputs: NOT_DATA
Node id: 3, inputs: [7], outputs: [0.6569866]
Node id: 4, inputs: [17], outputs: [-0.96139749]
Node id: 5, inputs: [11], outputs: [-0.99999021]
Node id: 6, inputs: [12], outputs: [-0.53657292]
Node id: 7 does not exist


### Real world example